In [ ]:
%matplotlib widget
from sympy import *
init_printing(use_latex=True)

# 22 - Plotting Module

## 22.1 - Plot’s Adaptive Algorithm

In [ ]:
A, t, wn, xi = symbols("A, t, omega_n, xi")
expr = (A * cos(sqrt(1 - xi**2) * wn * A * cos(t)) ) * exp(-xi * wn * t)
display(expr)
plot(expr.subs({wn: 0.5, xi: 0.55, A: 50}), (t, 0, 5))

In [ ]:
help(plot)

In [ ]:
plot(expr.subs({wn: 0.5, xi: 0.55, A: 50}), (t, 0, 5),
     adaptive=False,
     nb_of_points=300
)

## 22.2 - PlotGrid and Plot3d

In [ ]:
from sympy.plotting.plot import plot3d
x, y = symbols("x, y")
expr = cos(sqrt(x**2 + y**2) / 2) * exp(-sqrt(x**2 + y**2) / 10)
display(expr)r = -20
p3d = plot3d(expr, (x, -r, r), (y, -r, r),
             nb_of_points_x=100, nb_of_points_y=100,
             xlabel="x", ylabel="y", zlabel="f(x, y)")

In [ ]:
expr1 = expr.subs(y, 5)
p1 = plot(expr1, (x, -r, r), title="Plot at y=5", show=False)
expr2 = expr.subs(x, 3)
p2 = plot(expr2, (y, -r, r), title="Plot at x=3", show=False)
from sympy.plotting import PlotGrid
PlotGrid(2, 2, p1, p2, p3d)

## 23.4 - The LineOver1DRangeSeries class

In [ ]:
from sympy.plotting.plot import LineOver1DRangeSeries as L

In [ ]:
x = symbols("x")
line = L(sin(x), (x, 0, 2 * pi))

In [ ]:
line.get_segments()

In [ ]:
type(line.get_segments()[0][0])

In [ ]:
def get_xy(expr, r=None, **kwargs):
    if not r:
        s = list(expr.free_symbols)
        r = (s[0], -10, 10)
    if len(r) != 3:
        raise ValueError("r must represent a range of the form (symbol, start_value, end_value)")
    n = kwargs.pop("n", 300)
    kwargs.update({"nb_of_points": n})
    from sympy.plotting.plot import LineOver1DRangeSeries
    import numpy as np
    line = LineOver1DRangeSeries(expr, r, **kwargs)
    data = line.get_segments()
    x = np.zeros(len(data) + 1)
    y = np.zeros(len(data) + 1)
    x[0], y[0] = data[0][0]
    for i, d in enumerate(data):
        x[i + 1], y[i + 1] = d[1]
    return x, y

In [ ]:
A, B, t, wn, xi = symbols("A, B, t, omega_n, xi")
a = sqrt(1 - xi**2) * wn * t
expr1 = exp(-xi * wn * t)
expr2 = (A * cos(a) + B * sin(a)) * expr1
display(expr1, expr2)

In [ ]:
import matplotlib.pyplot as plt
d = { A: 1, B: 1, xi: 0.15, wn: 0.5 }
fig = plt.figure()
r = (t, 0, 50)
plt.plot(*get_xy(expr1.subs(d), r), linestyle="dashed", label="expr1")
plt.plot(*get_xy(expr2.subs(d), r), label="expr2")
plt.plot(*get_xy(expr2.subs(d), r, adaptive=False, n=30), ".", label="samples")
plt.xlim(r[1], r[2])
plt.ylim(-1.5, 1.5)
plt.title("Test Plot")
plt.xlabel("Time [s]")
plt.ylabel("Amplitude [m]")
plt.grid()
plt.legend()
plt.show()

## 22.5 - Improving the plot() function

In [ ]:
x = symbols("x")
plot((sin(x), (x, -10, 10)), (cos(x), (x, -5, 5)),
     legend=True, label=["sin", "cos"])

In [ ]:
from sympy_utils import pplot

In [ ]:
pplot((sin(x), (x, -10, 10)), (cos(x), (x, -5, 5)),
      legend=True, label=["sin", "cos"])

## 22.6 - Interactive Plots with ipywidgets

In [ ]:
A, t, wn, xi = symbols("A, t, omega_n, xi")
a = sqrt(1 - xi**2) * wn * t
expr1 = A * cos(a) * exp(-xi * wn * t)
expr1

In [ ]:
from ipywidgets import widgets
wn_slider = widgets.FloatSlider(
    value=0.5, min=0, max=1, step=0.01,
    description=r'$\omega_{n}$',
    continuous_update=False,
)
xi_slider = widgets.FloatSlider(
    value=0.15, min=0, max=1, step=0.01,
    description=r'$\xi$',
    continuous_update=False,
)
A_slider = widgets.FloatSlider(
    value=20, min=0, max=30, step=0.2,
    description='A',
    continuous_update=False,
)
vbox = widgets.VBox([A_slider, wn_slider, xi_slider])

In [ ]:
get_subs_dict = lambda: { 
    A: A_slider.value, 
    xi: xi_slider.value, 
    wn: wn_slider.value 
}

In [ ]:
import matplotlib.pyplot as plt
plt.ioff()
fig = plt.figure()
r = (t, 0, 50)
plt.xlim([r[1], r[2]])
plt.title("Test Plot")
plt.xlabel("Time [s]")
plt.ylabel("Amplitude [m]")
line1 = plt.plot(*get_xy(expr1.subs(get_subs_dict()), r), color="r")

def update_lines(change):
    x, y = get_xy(expr1.subs(get_subs_dict()), r)
    m, M = min(y), max(y)
    offset = 0.05 * (M - m)
    plt.ylim([m - offset, M + offset])
    line1[0].set_data(x, y)
    fig.canvas.draw()
    fig.canvas.flush_events()

A_slider.observe(update_lines, names='value')
wn_slider.observe(update_lines, names='value')
xi_slider.observe(update_lines, names='value')

widgets.AppLayout(
    header=vbox,
    center=fig.canvas,
    pane_heights=[1, 5, 0]
)

## 22. 7 - Creating a Custom Backend

In [ ]:
from sympy_utils import PlotlyBackend as PB

x = symbols("x")
plot(sin(x), cos(x), backend=PB, legend=True)

In [ ]:
from sympy.plotting.plot import plot3d
x, y = symbols("x, y")
r = sqrt(x**2 + y**2)
somb = 2 * bessely(1, pi * r) / (pi * r)
plot3d(somb, backend=PB)

In [ ]:
from sympy_utils import pplot
pplot(sin(x), cos(x), backend=PB, legend=True, label=["a", "b"])

In [ ]:
def plotly(*args, **kwargs):
    return pplot(*args, backend=PB, **kwargs)

In [ ]:
plotly(sin(x), cos(x))